# Hello Hybrid Jobs with Cuda-Q

[CUDA-Q](https://nvidia.github.io/cuda-quantum/latest/index.html) is a platform for hybrid quantum-classical computing. It offers a unified programming model designed for a hybrid setting for CPUs and GPUs. CUDA-Q contains support for programming in Python and in C++. 

In this notebook, you will learn how to run an Amazon Braket hybrid job with Cuda-Q using Bring Your Own Container (BYOC). This notebook assumes basic knowledge of Amazon Braket Hybrid Jobs. You can learn about Hybrid Jobs from [this page](https://docs.aws.amazon.com/braket/latest/developerguide/braket-what-is-hybrid-job.html) in the Amazon Braket Developer Guide. 

First, a job container is built with CUDA-Q and other GPU related settings configured. The procedure for BYOC is presented in this page from [Braker Developer Guide](https://docs.aws.amazon.com/braket/latest/developerguide/braket-jobs-byoc.html). The required files for building a container with CUDA-Quantum is the "container" folder, including
- Dockerfile: Describes how the container is built.
- requirements.txt: Additional Python dependencies to include.
- braket_container.py: The start up script of a job container (optional).

In addition, there is a shell script "container_build_and_push.sh" automates the procedure of pulling, building and pushing container images. The shell script takes two paramerers:
- image-name
- region-name

The shell script is called with the following syntax
```
container/container_build_and_push.sh <image-name> <region-name>

In [35]:
! container/container_build_and_push.sh braket-cudaq-byoc-job us-west-2

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
Login Succeeded
WARNING! Using --password via the CLI is insecure. Use --password-stdin.
Login Succeeded
#0 building with "desktop-linux" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 937B done
#1 DONE 0.0s

#2 [internal] load metadata for 292282985366.dkr.ecr.us-west-2.amazonaws.com/amazon-braket-pytorch-jobs:latest
#2 ...

#3 [auth] sharing credentials for 292282985366.dkr.ecr.us-west-2.amazonaws.com
#3 DONE 0.0s

#2 [internal] load metadata for 292282985366.dkr.ecr.us-west-2.amazonaws.com/amazon-braket-pytorch-jobs:latest
#2 DONE 0.6s

#4 [internal] load .dockerignore
#4 transferring context: 2B done
#4 DONE 0.0s

#5 [1/5] FROM 292282985366.dkr.ecr.us-west-2.amazonaws.com/amazon-braket-pytorch-jobs:latest@sha256:fe5f0c70d6b5cc587e24259c549aa1e221e0009885a065537028cbbd0193c06a
#5 DONE 0.0s

#6 [internal] load build context
#6 transferring context: 3

## Hybrid job with BYOC
Now, we have prepared the enviornment for Cuda-Q in a container image, let's run a BYOC job! First, we start will the necessary imports.

In [1]:
import numpy as np

from braket.jobs import hybrid_job
from braket.jobs.config import InstanceConfig
from braket.jobs.environment_variables import get_job_device_arn

We also prepare the URI of the container image. Fill the proper value of `aws_account_id`, `region_name` and `container_image_name` in the cell below. For example, with the shell command above, `region_name="us-west-2"` and `container_image_name="braket-cudaq-byoc-job"`. The cell below prints out the image URI.

In [2]:
aws_account_id = "537332306153"
region_name = "us-west-2"
container_image_name = "braket-cudaq-byoc-job"

# aws_account_id = "<aws-account-id>"
# region_name = "<region-name>"
# container_image_name = "<container-image-name>"

image_uri = f"{aws_account_id}.dkr.ecr.{region_name}.amazonaws.com/{container_image_name}:latest"
print(image_uri)

537332306153.dkr.ecr.us-west-2.amazonaws.com/braket-cudaq-byoc-job:latest


## Local job with CUDA-Q
We start by running a simulation job of a Bell circuit with CUDA-Q. The `hello_quantum` function in the code snippet below define the target backend to run the simulation. Then, it creates a Bell circuit and sample from the circuit. The string `qpp-cpu` in the `device` keyword argument is the name of a CUDA-Q CPU simulator. You can view the tutorial of CUDA-Q and the available backends in the [CUDA-Q documentation](https://nvidia.github.io/cuda-quantum/latest/index.html).

In [3]:
@hybrid_job(device='local:nvidia/qpp-cpu', image_uri=image_uri, local=True)
def hello_quantum():
    import cudaq

    # define the backend
    device=get_job_device_arn()
    cudaq.set_target(device.split('/')[-1])
    print("CUDA-Q backend: ", cudaq.get_target())

    # define the Bell circuit
    kernel = cudaq.make_kernel()
    qubits = kernel.qalloc(2)
    kernel.h(qubits[0])
    kernel.cx(qubits[0], qubits[1])

    # sample the Bell circuit
    result = cudaq.sample(kernel, shots_count=1000)
    measurement_probabilities = dict(result.items())
    print("Samples: ", measurement_probabilities)
    
    return measurement_probabilities

Skipping python version validation, make sure versions match between local environment and container.


When called, the decorated `hello_quantum` function starts a local job becuase of the keyword `local=True` in the hybrid_job decorator. The code inside the `hello_quantum` function will run locally in the environment defined by the container image that is built above.

In [4]:
hello_quantum()

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
Pulling docker container image. This may take a while.


Login Succeeded
latest: Pulling from braket-cudaq-byoc-job
Digest: sha256:76fc69d60fbb7fbee5f3cfaa0f370018546a73819e5ffdd09fb577fb124a4fec
Status: Image is up to date for 537332306153.dkr.ecr.us-west-2.amazonaws.com/braket-cudaq-byoc-job:latest
537332306153.dkr.ecr.us-west-2.amazonaws.com/braket-cudaq-byoc-job:latest



What's Next?
  View a summary of image vulnerabilities and recommendations → docker scout quickview 537332306153.dkr.ecr.us-west-2.amazonaws.com/braket-cudaq-byoc-job:latest


Boto3 Version:  1.35.22
Beginning Setup
Checking for Additional Requirements
Additional Requirements Check Finished
**** customer_module ****
['Path', 'PersistedJobDataFormat', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'clean_links', 'cloudpickle', 'get_input_data_dir', 'get_results_dir', 'hello_quantum', 'link_input', 'make_link', 'os', 'recovered', 'save_job_result']
Running Code As Process
CUDA-Q backend:  Target qpp-cpu
	simulator=qpp
	platform=default
	description=QPP-based CPU-only backend target
	precision=fp64

Samples:  {'11': 484, '00': 516}
Code Run Finished
80c6430a0ece55a73c4c1fb714c0e298bc73833f519c6dd8d7cc3bea87ba0fa9


## Remote jobs with CUDA-Q
By remove the `local=True` keyword argument (or, equivalently, set `local=False`), the job will run remotely.  

In [5]:
@hybrid_job(device='local:nvidia/qpp-cpu', image_uri=image_uri)
def hello_quantum():
    import cudaq
    
    device=get_job_device_arn()
    cudaq.set_target(device.split('/')[-1])
    print(cudaq.get_target())
    
    kernel = cudaq.make_kernel()
    qubits = kernel.qalloc(2)
    kernel.h(qubits[0])
    kernel.cx(qubits[0], qubits[1])
    
    result = cudaq.sample(kernel, shots_count=1000)
    measurement_probabilities = dict(result.items())
    print(measurement_probabilities)
    
    return measurement_probabilities

job = hello_quantum()
print(job.arn)

Skipping python version validation, make sure versions match between local environment and container.
arn:aws:braket:us-west-2:537332306153:job/81a601ae-093c-457f-9a12-7daa7ebc61b6


When called, the decorated `hello_quantum` function will create a Braket hybrid job on AWS, running the code defined in the decorated function with the environment specified by the container image built above.

In [6]:
result = job.result()
print(result)

{'11': 499, '00': 501}


## Summary
This notebooks show you how to build an container that support CUDA-Q in a BYOC job. The provided shell script automates the procedure of build a container image, simplifying the procedure to a one-line command. This notebooks use Bell circuit as an example to run a local and a remote Braket job on a CUDA-Q CPU backend.